# ミニプロジェクト / Miniproject

## ミニプロジェクト --- 太宰治か宮沢賢治か？著者を推定してみよう（基礎課題）

著者が分からない小説について、太宰治の作品か宮沢賢治の作品かを推定する手法を実装しましょう。  

このプロジェクトでは、[青空文庫](https://www.aozora.gr.jp/index.html) のテキストを使用します。
ただし、ルビや入力者注を削除するなど編集を加えています。 
また、両著者の小説の総文字数がおよそ同じになるように、太宰治の小説数を調整しています。

太宰治の小説が81、宮沢賢治の小説が148、未知の著者の小説が10与えられています。
太宰治と宮沢賢治の文章の傾向をそれぞれ調べて、未知の10の小説それぞれを太宰治と宮沢賢治のどちらが書いたかを推測します。
文章の傾向を評価する指標はいくつか提案されていますが、その中で文字n-gramに対する統計的指標を用います。  

例えば、宮沢賢治の小説では、文末が「です、ます」調であることが多いです。
一方、太宰は「である」調が多いです。
したがって、各著者の小説のbi-gramを集計すると、
宮沢賢治の小説では`です`や`す。`の出現確率が高く、太宰治の小説では`ある`や`る。`の出現確率が高くなります。
未知の著者の小説の`です`や`す。`の出現確率が`ある`や`る。`よりも高ければ、
著者は宮沢賢治ではないか？と予想することができそうです。

ですが、宮沢賢治の小説でも「である」調を使っているものがあります。
そこで、著者ごとに、すべての文字n-gramの出現確率の分布を求めて、
未知の著者の文字n-gram確率分布がどちらの著者のものに近いかを調べることにしましょう。

## Miniproject --- Guessing the author of a novel, 太宰治 or 宮沢賢治? (Basic exercises)

Let's implement a method that guesses who wrote a novel of an unknown author, 太宰治 (Osamu Dazai) or 宮沢賢治 (Kenji Miyazawa).

In this project, we use texts from [Aozora Bunko](https://www.aozora.gr.jp/index.html).
We have modified the texts by removing rubies and notes written by those who typed the texts.
We have also adjusted the number of novels by 太宰治 so that
the total numbers of letters in novels of the two authors are nearly equal.

We provide 81 novels by 太宰治, 148 novels by 宮沢賢治, and 10 novels by unknown authors.
Observe the characteristics of the texts by 太治宰 and 宮沢賢治, and for each of the novels by unknown authors,
guess who wrote it, 太宰治 or by 宮沢賢治.
Among the known features for classifying texts, try to use statistical features of letter n-grams.

For example, in the novels by 宮沢賢治, sentences tend to end with the です-ます style.
On the other hand, in the novels by 太宰治, sentences tend to end with the である style.
Therefore, given that the bi-grams in the novels by each author have been counted,
the probability of occurrences of `です` or ` す。` should be high in the novels by 宮沢賢治,
and that of `ある` or `る。` should be high in those by 太宰治.
If the probability of `です` or ` す。` in the novel by an unknown author is higher than that of `ある` or `る。`,
you can then conclude that the author is 宮沢賢治.

However, some novels by 宮沢賢治 are written in the です-ます style.
Therefore, compute the probability distribution of all the letter n-grams of each of the authors,
and observe the similarity between the probability distribution of letter n-grams of an unknown author
and that of 宮沢賢治 (太宰治).

## 準備

小説は、表データとして保存されています。
表の列名は順に`author`・`title`・`text`であり、それぞれ著者名・題名・本文が入っています。
表の各行は1つの小説の情報を格納し、その`author`列の値によって、その小説の著者を特定できます。

以下の課題1と課題5では、このような表データの`DataFrame`オブジェクトが関数に渡されます。

尚、`DataFrame`から取り出された1列は、for文で反復処理できます。例えば、
```Python
for text in novels['text']:
    print(text)
```
とすると、小説の表データ `novels` から、本文を順に印字できます。

## Preparation

A dataset of novels is stored into a table.
The table of novel data consists of the columns named `author`, `title`, and `text`,
which contain author names in Japanese, titles, and texts of novels, respectively.
Each row of the table stores the information of one novel;
we can identify the author of the novel by the `author` value.  

In the following Exercises 1 and 5, such a table of type `DataFrame` is given to functions.

Note that for-statements can handle iterations over a column extracted from `DataFrame`. 
For example, the following for-statement
```Python
for text in novels['text']:
    print(text)
```
prints the texts of all the novels stored in `novels`.

第3回で、n-gram の求め方を学びました。
以下の関数 `multiline_ngrams(n,text)` は、与えられたテキスト `text` に含まれるn-gramのリストを返します。

In the 3rd lecture, you learned how to compute n-grams.
The following function `multiline_ngrams(n,text)` computes a list of n-grams that appear in the given text `text`.

In [146]:
def multiline_ngrams(n, text):
    l = []
    for sentence in text.split('\n'):
        for i in range(len(sentence)-n+1):
            l.append(sentence[i:i+n])
    return l

以下の課題では、次のジェネレータを用いてもよいです。

You can also use the following generator in the following exercises.

In [147]:
def multiline_ngrams_gen(n, text):
    for sentence in text.split('\n'):
        for i in range(len(sentence)-n+1):
            yield sentence[i:i+n]

## 課題１：n-gramの抽出

引数 `novels` に入っている小説のうち、`author` が書いた小説に現れるすべてのn-gramのリストもしくはイテレータを返す関数`author_ngrams(n, author, novels)` を定義してください。
引数 `author` に著者名（太宰治か宮沢賢治）が渡されます。

各n-gramは、著者のすべての小説に現れる回数だけリストもしくはイテレータに現れなければなりません。

## Exercise 1: Extraction of n-grams

Define a function `author_ngrams(n,author,novels)` which returns a list or iterator of all n-grams
that appear in the texts in `novels` each of which is written by `author`.
The parameter `author` is 太宰治 or 宮沢賢治.

Each n-gram must appear in the list or iterator as many times as it appears in all the novels by the author.

In [148]:
##########################################################
##  <[ project1-1-author_ngrams ]> 解答セル (Answer cell)
##  このコメントの書き変えを禁ず (Never edit this comment)
##########################################################

def author_ngrams(n, author, novels):
    l = []
    novels = novels[novels.author == author]
    for text in novels['text']:
        l += (multiline_ngrams(n, text))
    return l

提出前に以下のテストセルを実行し、 `True` のみが出力されることを確認してください。

Before submission, execute the following test cell and check if only `True` is printed.

In [149]:
def tester():
    import pandas as pd
    TEST_NOVELS = [['太宰治', '政治家と家庭', '頭の禿げた善良そうな記者君が何度も来て、書け書け、と頭の汗を拭きながらおっしゃるので、書きます。\n佐倉宗五郎子別れの場、という芝居があります。ととさまえのう、と泣いて慕う子を振り切って、宗五郎は吹雪の中へ走って消えます。あれを、どうお思いでしょうか。アメリカ人が見たら、あれをどう感ずるでしょうか。ロシヤ人が見たら、何と判断するでしょうか。\nしかし私たち日本人、殊に男が何か仕事に打ち込んだ場合、たいていこの宗五郎のようになってしまいます。\n家族は、捨ててよいものでしょうか。日本の政治家たちは、たいてい家庭を捨てているようです。ひどいのになると、独身だか妻帯者だか、わからない人物もあります。しつけの良い家庭を営んでいる政治家は、少いように思われます。\nしつけのよい家庭を維持しながら、よい仕事も出来るという政治家もあってよいと思います。これこそ、至難の事業であります。けれども、兄は、それが出来るかも知れない極めて少数のひとの一人だと思います。\n無理なお願いでしょうけれどもお願いしてみます。私の為のお願いではありません。\n'],
                   ['宮沢賢治', '会計課', '九時六分のかけ時計\nその青じろき盤面に\nにはかに雪の反射来て\nパンのかけらは床に落ち\nインクの雫かわきたり\n'],
                  ]
    TEST_NOVELS = pd.DataFrame(TEST_NOVELS, columns=['author', 'title', 'text'])
    small_2_dazai = list(author_ngrams(2, '太宰治', TEST_NOVELS))
    small_2_miyazawa = list(author_ngrams(2, '宮沢賢治', TEST_NOVELS))
    print(len(small_2_dazai) == 452)
    print(len(small_2_miyazawa) == 44)
    print(small_2_dazai.count('す。') == 11)
    print(small_2_miyazawa.count('のか') == 2)

    novels = pd.read_csv('known_novels.csv', encoding='utf-8')
    large_3_dazai = list(author_ngrams(3, '太宰治', novels))
    large_3_miyazawa = list(author_ngrams(3, '宮沢賢治', novels))
    print(len(large_3_dazai) == 899275)
    print(len(large_3_miyazawa) == 868498)
    print(large_3_dazai.count('である') == 2891)
    print(large_3_miyazawa.count('である') == 290)
tester()

True
True
True
True
True
True
True
True


## 課題２：n-gramの出現回数

n-gramのリストもしくはイテレータが与えられたとき、各々のn-gramをキーとして、
その出現回数を値（バリュー）とする辞書を返す関数 `histogram(ngs)` を定義してください。
引数 `ngs` には、n-gramのリストもしくはイテレータが与えられます。

### 注意

各n-gramに対するループの中で、さらに各n-gramを調べる、といった二重ループを書くと、
実行時間が非常に大きくなる可能性があります。
3-2「for文の計算量」を参照してください。
少なくとも、この課題では二重のループは必要ありません。

## Exercise 2: Occurrences of n-grams

Define a function `histogram(ngs)`,
which is given a list or iterator of n-grams and returns a dictionary whose keys are n-grams
with the number of their occurrences as their value.
The parameter `ngs` is a list or iterator of n-grams.

### Attention

If you write a nested loop that tests each n-gram in an outer loop that also tests each n-gram,
then it will take a very long time to execute.
Refer to 3-2 (complexity of for statements).
In this exercise, no nested loop is necessary.

In [150]:
##########################################################
##  <[ project1-2-histogram ]> 解答セル (Answer cell)
##  このコメントの書き変えを禁ず (Never edit this comment)
##########################################################

def histogram(ngs):
    ans = {}
    for key in ngs:
        ans[key] = 0
    for key in ngs:
        ans[key] += 1
    return ans

提出前に以下のテストセルを実行し、 `True` のみが出力されることを確認してください。

Before submission, execute the following test cell and check if only `True` is printed.

In [151]:
def tester():
    import pandas as pd
    novels = pd.read_csv('known_novels.csv', encoding='utf-8')
    dazai_histogram = histogram(author_ngrams(3, '太宰治', novels))
    miyazawa_histogram = histogram(author_ngrams(3, '宮沢賢治', novels))
    unknown_novels = pd.read_csv('unknown_novels.csv', encoding='utf-8')
    un0_histogram = histogram(multiline_ngrams(3, unknown_novels.loc[0,'text']))
    print(len(dazai_histogram) == 268576)
    print(len(miyazawa_histogram) == 245770)
    print(dazai_histogram['である'] == 2891)
    print(miyazawa_histogram['である'] == 290)
    print(dazai_histogram['です。'] == 1203)
    print(miyazawa_histogram['です。'] == 1875)
    print(un0_histogram['である'] == 4)
    print(un0_histogram['です。'] == 18)
tester()

True
True
True
True
True
True
True
True


## 課題３：n-gramの確率分布

課題２で定義した `histogram` によって算出されたn-gram出現回数の分布 `hist` が与えられたら、
n-gramの確率分布を返す関数 `probability_distribution(hist)` を定義してください。
各n-gramの出現回数を、全n-gramの出現回数の総和で割ればよいです。
関数 `probability_distribution` は、n-gramをキーとしてその出現の確率を値とする辞書を返します。

## Exercies 3: Probability ditributions of n-grams

Define a function `probability_distribution(hist)`,
which is given a distribution of occurrences of n-grams `hist` computed by the function `histogram` in Exercise 2,
and returns the probability distribution of n-grams.
The number of occurrences of each n-gram is divided by the total number of occurrences of all the n-grams.
The function `probability_distribution` returns a dictionary
whose keys are n-grams with the probability of their occurrences as their value.

In [152]:
##########################################################
##  <[ project1-3-probability_distribution ]> 解答セル (Answer cell)
##  このコメントの書き変えを禁ず (Never edit this comment)
##########################################################

def probability_distribution(hist):
    hist_num = 0
    for key, val in hist.items():
        hist_num += val
    hist_ans = {}
    for key, val in hist.items():
        hist_ans[key] = val / hist_num
    return hist_ans

提出前に以下のテストセルを実行し、 `True` のみが出力されることを確認してください。

Before submission, execute the following test cell and check if only `True` is printed.

In [153]:
def tester():
    import pandas as pd
    novels = pd.read_csv('known_novels.csv', encoding='utf-8')
    dazai_histogram = histogram(author_ngrams(3, '太宰治', novels))
    miyazawa_histogram = histogram(author_ngrams(3, '宮沢賢治', novels))
    print(round(probability_distribution(dazai_histogram)['である']*10**8) == 321481)
    print(round(probability_distribution(miyazawa_histogram)['である']*10**8) == 33391)
tester()

True
True


## 課題４：n-gramの確率分布間の距離

いよいよ異なる文書におけるn-gramの確率分布の間の距離を計算していきます。
$d_1$ と $d_2$ という二つのn-gram確率分布が与えられたとしましょう。
以下の数式では、n-gram $x$ に対して $d_i(x)$ は $d_i$ における $x$ の確率を表します。

$d_1$ と $d_2$ のTankard距離は、各々のn-gramの二つのテキストにおける出現確率の差の総和です。
同じn-gramの確率の差が大きければ大きいほど、二つの文書は異なると考えられます。
すべてのn-gramに対してその差の平均を求めます。
したがって、Tankard距離は次のように定義されます。 

$\mbox{Tankard}(d_1, d_2) = 
\frac{1}{\mbox{card}(C)} \sum_{x \in C} {|d_1(x) - d_2(x)|}$

ここで、$C$ は $d_1$ と $d_2$ の両方で確率が正になるn-gramの集合を表していて、以下のように定義されます。

$C = \{~x~|~d_1(x)>0~\mbox{かつ}~d_2(x) > 0 \}$

$\mbox{card}(C)$ は集合 $C$ の要素数を表します。

n-gram確率分布が辞書で表されているとき、辞書に登録されていないn-gramの確率は0と考えます。

では、二つのn-gram確率分布が辞書 `d1` と `d2` として与えられたとき、それらの間のTankard距離を返す関数 `Tankard(d1,d2)` を定義してください。

## Exercise 4: Distance between probability distributions of n-grams

You now compute the distance between probability distributions of n-grams in different texts.
Suppose that two n-gram probability distributions $d_1$ and $d_2$ are given.
In the following mathematical expressions, $d_i(x)$ denotes the probability of an n-gram $x$ in $d_i$.

The Tankard distance between $d_1$ and $d_2$ is obtained by summing the difference of the probabilities
of each n-gram in the two texts.
If the difference of the probabilities of each n-gram is larger, the two texts are considered more different.
The average difference for all n-grams is then computed.
The Tankard distance is therefore defined as follows.

$\mbox{Tankard}(d_1, d_2) =
\frac{1}{\mbox{card}(C)} \sum_{x \in C} {|d_1(x) - d_2(x)|}$,

in which $C$ denotes the set of n-grams whose probabilities in $d_1$ and $d_2$ are both positive, that is,

$C = \{~x~|~d_1(x)>0~\mbox{ and }~d_2(x) > 0 \}$,

and $\mbox{card}(C)$ denotes the number of elements in $C$.

If an n-gram probability distribution is represented by a dictionary,
the probability of an n-gram that is not stored in the dictionary is considered 0.

Now, define a function `Tankard(d1,d2)` that returns the Tankard distance
between the two n-gram probability distributions `d1` and `d2` that are given as dictionaries.

In [154]:
##########################################################
##  <[ project1-4-Tankard ]> 解答セル (Answer cell)
##  このコメントの書き変えを禁ず (Never edit this comment)
##########################################################

def Tankard(d1, d2):
    diff_sum = 0
    ele_num = 0
    for key, val in d1.items():
        if key in d2:
            if val > 0 and d2[key] > 0:
                diff_sum += abs(val - d2[key])
                ele_num += 1
    return diff_sum / ele_num

提出前に以下のテストセルを実行し、 `True` のみが出力されることを確認してください。

Before submission, execute the following test cell and check if only `True` is printed.

In [155]:
def tester():
    import pandas as pd
    novels = pd.read_csv('known_novels.csv', encoding='utf-8')
    dazai_histogram = histogram(author_ngrams(3, '太宰治', novels))
    miyazawa_histogram = histogram(author_ngrams(3, '宮沢賢治', novels))
    print(round(Tankard(probability_distribution(dazai_histogram),probability_distribution(miyazawa_histogram))*10**8) == 855)
tester()

True


## 課題５：著者の推定

著者が未知の小説のそれぞれに対して、太宰治か宮沢賢治のどちらが書いたかを推定しましょう。
関数 `which_author(n, known_novels, unknown_novels)` を定義してください。それは、
* 正の整数 `n`・既知の小説データ `known_novels`・未知の小説データ `unknown_novels` を受け取り、
* `unknown_novels`に含まれる各小説について著者を推定して、結果を順にリストで返します。

推定方法は次の通りです。
* 対象の小説のn-gramの確率分布を計算し、
* それと`known_novels`内の太宰治の小説のn-gramの確率分布とのTankard距離と、
* 同様に宮沢賢治との距離を求め、
* 太宰治との距離が宮沢賢治の方よりも小さければ、文字列 `'太宰治'` を結果とし、
* そうでなければ、文字列 `'宮沢賢治'` を結果とします。

尚、未知の小説データ `unknown_novels` には、`author`列に著者名（真の解）を含みますが、これを推定に使ってはなりません。

## Exercise 5: Guessing the author

Now, let's guess who wrote unknown novels, 太宰治 or 宮沢賢治.
Define a function `which_author(n, known_novels, unknown_novels)` that
* takes a positive integer `n`, a dataset of known novels `known_novels`, and a dataset of unknown novels `unknown_novels`, and 
* returns a list of guessed results for all the novels in `unknown_novels` in order. 

The author of a novel is guessed as follows:
* Calculate the n-gram probability distribution of a given unknown novel;
* Calculate its Tankard distance to that of the novels of 太宰治 in `known_novels`;
* Calculate its Tankard distance to that of 宮沢賢治 in `known_novels`;
* Conclude that the author is `'太宰治'` if the distance to 太宰治 is smaller than that to 宮沢賢治,
* Conclude that the author is `'宮沢賢治'` otherwise.

Note that `unknown_novels` contains author information (i.e., true solutions) in the `author` column, but you are not allowed to use it for guessing.

In [156]:
##########################################################
##  <[ project1-5-which_author ]> 解答セル (Answer cell)
##  このコメントの書き変えを禁ず (Never edit this comment)
##########################################################

def which_author(n, known_novels, unknown_novels):
    ans = []
    dazai_ngrams = author_ngrams(n, "太宰治", known_novels)
    dazai_hist = histogram(dazai_ngrams)
    dazai_d = probability_distribution(dazai_hist)

    miyazawa_ngrams = author_ngrams(n,  "宮沢賢治", known_novels)
    miyazawa_hist = histogram(miyazawa_ngrams)
    miyazawa_d = probability_distribution(miyazawa_hist)

    ngram_l = []
    for text in unknown_novels["text"]:
        ngram_l = multiline_ngrams(n, text)
        unknown_hist = histogram(ngram_l)
        unknown_d = probability_distribution(unknown_hist)

        dazai_unknown_distance = Tankard(dazai_d, unknown_d)
        miyazawa_unknown_distance = Tankard(miyazawa_d, unknown_d)

        if dazai_unknown_distance < miyazawa_unknown_distance:
            ans.append("太宰治")
        else:
            ans.append("宮沢賢治")
    return ans

提出前に以下のテストセルを実行し、 `True` のみが出力されることを確認してください。

Before submission, execute the following test cell and check if only `True` is printed.

In [157]:
def tester():
    import pandas as pd
    known_novels = pd.read_csv('known_novels.csv', encoding='utf-8')
    unknown_novels = pd.read_csv('unknown_novels.csv', encoding='utf-8')
    print(which_author(3, known_novels, unknown_novels) == ['太宰治', '太宰治', '太宰治', '太宰治', '太宰治', '宮沢賢治', '宮沢賢治', '宮沢賢治', '宮沢賢治', '宮沢賢治'])
tester()

True
